In [ ]:


from keras import layers
from keras import backend as K
from keras.models import Model
from keras.models import load_model
#from tensorflow.python.keras.utils import CustomObjectScope
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Input, Reshape, Add
from keras.utils.generic_utils import get_custom_objects
from keras.regularizers import l1_l2, l2, l1
from keras.metrics import categorical_accuracy
from tensorflow.keras.backend import eval


from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from sklearn.model_selection import train_test_split
from collections import Counter
from scipy.optimize import nnls

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import glob
import os
import sys
import subprocess
import math
import time
import random
import socket


import innvestigate


In [ ]:
target_dir = 'fuzzgoat' #Make this customizable for the future

IP = "127.0.0.1"
PORT = 4455
ADDR = (IP, PORT)
SIZE = 1024
FORMAT = "iso-8859-1"

checkpoint_path = './targets/' + target_dir + '/inc_learn/model.hd5'

seed_list = glob.glob('./targets/' + target_dir + '/out/queue/id*') #Note that you will need seperate input and output directories between the two times the fuzzer is run
new_seeds = glob.glob('./targets/' + target_dir + '/out/queue/id*') #Will be relevent for incremental learning, no need for it currently
print('done')

argvv = sys.argv[1:] #This is supposed to give you the name of the program you are running (.exe)
call = subprocess.check_output
print(sys.argv)

os.path.isdir("./targets/" + target_dir + "/bitmaps/") or os.makedirs("./targets/" + target_dir + "/bitmaps/")
os.path.isdir("./targets/" + target_dir + "/in/gen/") or os.makedirs("./targets/" + target_dir + "/in/gen/")
os.path.isdir("./targets/" + target_dir + "/checkpoint") or os.makedirs("./targets/" + target_dir + "/checkpoint")
os.path.isdir("./targets/" + target_dir + "/out/queue_grads") or os.makedirs("./targets/" + target_dir + "/out/queue_grads")
os.path.isdir("./targets/" + target_dir + "/inc_learn") or os.makedirs("./targets/" + target_dir + "/inc_learn")

print(seed_list)

In [ ]:
print(sys.argv)

Setting up bitmaps and getting data (Courtesy of Neuzz)

In [ ]:
def preprocessing():
    global MAX_BITMAP_SIZE
    global MAX_FILE_SIZE
    global seed_list
    global new_seeds
    global counter
    global label
    global u_idx
    
    
    seed_list = glob.glob('./targets/' + target_dir + '/out/queue/id*')
    seed_list.sort()
    SPLIT_RATIO = len(seed_list)
    rand_index = np.arange(SPLIT_RATIO)
    np.random.shuffle(seed_list)
    new_seeds = glob.glob('./targets/' + target_dir + '/out/queue/id*') #Once again will be relevent later for incremental learning
    
    #cwd = os.getcwd()
    
    max_file_name = call(['ls', '-S', './targets/' + target_dir + '/out/queue/']).decode('utf8').split('\n')[0].rstrip('\n')
    MAX_FILE_SIZE = os.path.getsize('./targets/' + target_dir + '/out/queue/' + max_file_name)
    #print(seed_list)
    
        # obtain raw bitmaps
    raw_bitmap = {}
    tmp_cnt = []
    out = ''
    for file in seed_list:
        tmp_list = []
        try:
            # append "-o tmp_file" to strip's arguments to avoid tampering tested binary.
            if argvv[0] == './strip':
                out = call(['./afl-showmap', '-q', '-e', '-o', '/dev/stdout', '-m', '512', '-t', '500'] + ['./targets/' + target_dir + '/' + target_dir] + [file] + ['-o', 'tmp_file'])
                #out = call(['./afl-showmap', '-q', '-e', '-o', '/dev/stdout', '-m', '512', '-t', '500'] + argvv + [file] + ['-o', 'tmp_file'])
            else:
                out = call(['./afl-showmap', '-q', '-e', '-o', '/dev/stdout', '-m', '512', '-t', '500'] + ['./targets/' + target_dir + '/' + target_dir] + [file])
                #print(out)
                #out = call(['./afl-showmap', '-q', '-e', '-o', '/dev/stdout', '-m', '512', '-t', '500'] + argvv + [file])
        except subprocess.CalledProcessError:
            print("find a crash")
        #print(out)
        #print("end")
        for line in out.splitlines():
            edge = line.split(b':')[0]
            #print(edge)
            tmp_cnt.append(edge)
            tmp_list.append(edge)
        raw_bitmap[file] = tmp_list
        #print(raw_bitmap)
    counter = Counter(tmp_cnt).most_common()
    #print("Counter:")
    #print(counter)
    #print(raw_bitmap)

    # save bitmaps to individual numpy label
    label = [int(f[0]) for f in counter]
    #print(label)
    bitmap = np.zeros((len(seed_list), len(label)))
    for idx, i in enumerate(seed_list):
        #print(idx)
        tmp = raw_bitmap[i]
        for j in tmp:
            #print(j)
            if int(j) in label:
                bitmap[idx][label.index((int(j)))] = 1

    # label dimension reduction
    fit_bitmap, u_idx = np.unique(bitmap, axis=1, return_index=True)
    #print("data dimension" + str(fit_bitmap.shape))
    #print("data dimension" + str(bitmap.shape))

    #print(fit_bitmap)    
    #print(bitmap)
    # save training data
    MAX_BITMAP_SIZE = fit_bitmap.shape[1]
    for idx, i in enumerate(seed_list):
        file_name = "./targets/" + target_dir + "/bitmaps/" + i.split('/')[-1]
        np.save(file_name, fit_bitmap[idx])

print('done')

In [ ]:
preprocessing()

Accuracy Metric (Jaccard Similarity Index)

In [ ]:
# compute jaccard accuracy for multiple label
def jaccard(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)
    y_true = tf.round(y_true)
    y_pred = tf.round(y_pred)
    matching = tf.cast(tf.math.count_nonzero(tf.equal(y_true,y_pred)), tf.float64)
    denom = tf.cast(tf.math.add(tf.size(y_true), tf.size(y_pred)), tf.float64)
    denom2 = tf.cast(tf.math.subtract(denom, matching), tf.float64)
    return tf.cast(tf.divide(matching, denom2), tf.float64)

In [ ]:
#prepare input into usable vectors (dehex-ifying  the file and storying it as a numerical array)
def vectorize(f):
    seed = np.zeros((1, MAX_FILE_SIZE))
    tmp = open(f, 'rb').read()
    ln = len(tmp)
    if ln < MAX_FILE_SIZE:
        tmp = tmp + (MAX_FILE_SIZE - ln) * b'\x00'#adding empty bytes to fill up smaller files
    seed[0] = [j for j in bytearray(tmp)]
    seed = seed.astype('float64')
    np.set_printoptions(threshold=sys.maxsize)
    print(seed)
    seed = seed / 255 #normalize with 255 because maximum hex value is FF = 255
    print(seed)
    
    #print(tmp)
    
    #loading in bitmap
    file_name = "./targets/" + target_dir + "/bitmaps/" + f.split('/')[-1] + ".npy"
    bitmap = np.load(file_name)
    return seed, bitmap


In [ ]:
#prepare input into usable vectors (dehex-ifying  the file and storying it as a numerical array)
def vectorize_range(l, h):
    
    seeds = np.zeros((h - l, MAX_FILE_SIZE))
    bitmaps = np.zeros((h - l, MAX_BITMAP_SIZE))
    
    for i in range(l, h):
        seed, bitmap = vectorize(seed_list[i])
        seeds[i-l] = seed
        bitmaps[i-l] = bitmap
    
    return seeds, bitmaps


In [ ]:
vectorize(seed_list[11])

In [ ]:
def prepare_data():
    X, y = [[],[]]
    for i in seed_list:
        tmp = vectorize(i)
        X.append([tmp[0]])
        y.append([tmp[1]])
        
    print(len(X))
    print(len(y))
    return train_test_split(X, y, test_size=0.2, random_state=1) #Want to preserve this split in case for live updates


In [ ]:
class LearningRateSchedulerPerBatch(LearningRateScheduler):
    """ Callback class to modify the default learning rate scheduler to operate each batch"""
    def __init__(self, schedule, verbose=0):
        super(LearningRateSchedulerPerBatch, self).__init__(schedule, verbose)
        self.count = 0  # Global batch index (the regular batch argument refers to the batch index within the epoch)

    def on_epoch_begin(self, epoch, logs=None):
        pass

    def on_epoch_end(self, epoch, logs=None):
        pass

    def on_batch_begin(self, batch, logs=None):
        super(LearningRateSchedulerPerBatch, self).on_epoch_begin(self.count, logs)

    def on_batch_end(self, batch, logs=None):
        super(LearningRateSchedulerPerBatch, self).on_epoch_end(self.count, logs)
        self.count += 1
        
        
class ModelCheckpointEnhanced(ModelCheckpoint):
    def __init__(self, *args, **kwargs):
        # Added arguments
        self.callbacks_to_save = kwargs.pop('callbacks_to_save')
        self.callbacks_filepath = kwargs.pop('callbacks_filepath')
        super().__init__(*args, **kwargs)

    def on_epoch_end(self, epoch, logs=None):
        # Run normal flow:
        super().on_epoch_end(epoch,logs)

        # If a checkpoint was saved, save also the callback
        filepath = self.callbacks_filepath.format(epoch=epoch + 1, **logs)
        if self.epochs_since_last_save == 0 and epoch!=0:
            if self.save_best_only:
                current = logs.get(self.monitor)
                if current == self.best:
                    # Note, there might be some cases where the last statement will save on unwanted epochs.
                    # However, in the usual case where your monitoring value space is continuous this is not likely
                    with open(filepath, "wb") as f:
                        pickle.dump(self.callbacks_to_save, f)
            else:
                with open(filepath, "wb") as f:
                    pickle.dump(self.callbacks_to_save, f)

In [ ]:
def gen_model():
    batch_size = 32
    num_classes = MAX_BITMAP_SIZE
    epochs = 50

    model = Sequential()
    model.add(Dense(1000, input_dim=MAX_FILE_SIZE))
    model.add(Activation('relu'))
    model.add(Dense(num_classes))
    model.add(Activation('sigmoid'))

    #opt = tf.keras.optimizers.adam(lr=0.0001)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[jaccard])
    model.summary()

    return model

In [ ]:
def train(model):
    X_train, X_val, y_train, y_val = prepare_data()
    X_train = np.array(X_train).reshape(-1, MAX_FILE_SIZE)
    y_train = np.array(y_train).reshape(-1, MAX_BITMAP_SIZE)
    X_val = np.array(X_val).reshape(-1, MAX_FILE_SIZE)
    y_val = np.array(y_val).reshape(-1, MAX_BITMAP_SIZE)
    
    print(X_train.shape)
    print(MAX_FILE_SIZE)
    
   # ckpt_callback = ModelCheckpoint(filepath='./targets/' + target_dir + '/inc_learn/model.h5', monitor='val_loss')
   # lr_decay_callback = LearningRateSchedulerPerBatch(
   #                lambda step: ((learning_rate - min_learning_rate) * decay_rate ** step + min_learning_rate))

    #callbacks_list = [ckpt_callback, lr_decay_callback]
    print(X_train.shape)
    #Option insert learning decay rate here
    his = model.fit(X_train, y_train, validation_data = (X_val, y_val),  epochs = 50, batch_size=32) #treating the entire X-train as one input rn?????
    # Save model and weights
    model.save_weights("./targets/" + target_dir + "model.h5")
    return model, his

In [ ]:
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.7
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((1 + epoch) / epochs_drop))
    return lrate

In [ ]:
learning_rate = 0.002
min_learning_rate = 0.0001
decay_rate = 0.95

In [ ]:
def train_one(model):
    X_train, X_val, y_train, y_val = prepare_data()
    X_train = np.array(X_train).reshape(-1, MAX_FILE_SIZE)
    y_train = np.array(y_train).reshape(-1, MAX_BITMAP_SIZE)
    X_val = np.array(X_val).reshape(-1, MAX_FILE_SIZE)
    y_val = np.array(y_val).reshape(-1, MAX_BITMAP_SIZE)
    
    print(X_train.shape)
    print(MAX_FILE_SIZE)
    
    ckpt_callback = ModelCheckpoint(filepath='./targets/' + target_dir + '/inc_learn/model.h5', monitor='val_loss')
    lr_decay_callback = LearningRateSchedulerPerBatch(
                    lambda step: ((learning_rate - min_learning_rate) * decay_rate ** step + min_learning_rate))

    callbacks_list = [ckpt_callback, lr_decay_callback]
    print(X_train.shape)
    #Option insert learning decay rate here
    for i in range(len(X_train)):
        his = model.fit(X_train[i].reshape(1, MAX_FILE_SIZE), y_train[i].reshape(1, MAX_BITMAP_SIZE), validation_data = (X_val, y_val),  epochs = 50, callbacks=callbacks_list, batch_size=32) #treating the entire X-train as one input rn?????
    # Save model and weights
    model.save_weights("./targets/" + target_dir + "model.h5")
    return model, his

In [ ]:
def build_model():
    
    model = gen_model()
    his = train(model)
    
    return model, his

In [ ]:
model, his = build_model()

In [ ]:
def inc_model(x, y):
    # Initialize Callbacks:
    lr_decay_callback = LearningRateSchedulerPerBatch(
                        lambda step: ((learning_rate - min_learning_rate) * decay_rate ** step + min_learning_rate))
    # Modified Checkpoint Callback
    ckpt_callback = ModelCheckpointEnhanced(filepath='./targets/' + target_dir + '/inc_learn/model.h5', monitor='val_loss',
                                    callbacks_to_save=lr_decay_callback ,callbacks_filepath='./checkpoint/LRcallback.{epoch:02d}.pickle' )

    callbacks = [ckpt_callback, lr_decay_callback]

    # Load checkpoint:
    if checkpoint_path is not None:
        # Load model:
        model = load_model(checkpoint_path, custom_objects={"jaccard": jaccard})
        # Finding the epoch index from which we are resuming
        initial_epoch = get_init_epoch(checkpoint_path)
        # loading the callback from pickle
        loaded_callback = pickle.load(open( callback_checkpoint_path, "rb" ))
        # Update the callback instance
        callbacks[1] = loaded_callback
    else:
        model = gen_model()
        initial_epoch = 0
    # Start/resume training
    model.fit(x, y, callbacks=callbacks, initial_epoch=initial_epoch)


In [ ]:
X_train, X_val, y_train, y_val = prepare_data()
X_train = np.array(X_train).reshape(-1, MAX_FILE_SIZE)
y_train = np.array(y_train).reshape(-1, MAX_BITMAP_SIZE)
X_val = np.array(X_val).reshape(-1, MAX_FILE_SIZE)
y_val = np.array(y_val).reshape(-1, MAX_BITMAP_SIZE)

In [ ]:
print(X_val[0].shape)

In [ ]:
f = plt.figure(figsize=(20,8))

#accuracy
plt1 = f.add_subplot(121)
plt1.plot(his.history['jaccard'], label = str('Training accuracy'))
plt1.plot(his.history['val_jaccard'], label = str('Validation accuracy'))
plt.legend()
plt.title('accuracy')

#loss
plt2 = f.add_subplot(122)
plt2.plot(his.history['loss'], label = str('Training loss'))
plt2.plot(his.history['val_loss'], label = str('Validation loss'))
plt.legend()
plt.title('loss')

plt.show()

In [ ]:
def train_generate(batch_size):
    global seed_list
    while 1:
        np.random.shuffle(seed_list)
        # load a batch of training data
        for i in range(0, len(seed_list), batch_size):
            # load full batch
            if (i + batch_size) > len(seed_list):
                x, y = vectorize_range(i, len(seed_list))
                x = x.astype('float32') / 255
            # load remaining data for last batch
            else:
                x, y = vectorize_range(i, i + batch_size)
                x = x.astype('float32') / 255
            yield (x, y)

In [ ]:
def inc_model():
    loss_history = LossHistory()
    lrate = keras.callbacks.LearningRateScheduler(step_decay)
    callbacks_list = [loss_history, lrate]
    model.fit_generator(train_generate(16),
                        steps_per_epoch=(len(seed_list) / 16 + 1),
                        epochs=50,
                        verbose=1, callbacks=callbacks_list)
    # Save model and weights
    model.save_weights("hard_label.h5")

In [ ]:
inc_model()

In [ ]:
def construct_gan(verbose = False):
    disc_model = dist_model()
    train(disc_model)
    
    gen_model = gen_model()
    
    gan = Sequential()
    gan.add(gen_model)
    gan.add(disc_model)
    
    disc_model.trainable = False
    
    gan.compile(loss='binary_crossentropy', optimizer='adam', metrics=[jaccard])

Idea: Run the intial neural network in reverse to generate desired edge coverage patterns

In [ ]:

#model2 = tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)

#Reversing the prebuilt model
sample_map = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.])
input_map = sample_map



for i in range(len(input_map)):
    if input_map[i] == 0:
        input_map[i] += 1e-5
    if input_map[i] == 1:
        input_map[i] -= 1e-5
       
#input_map = np.array(input_map)
#print(input_map)
#Working through inverting sigmoid acitvation
inv_sig = tf.subtract(1.0, input_map)
inv_sig = tf.divide(input_map, inv_sig)
inv_sig = tf.math.log(inv_sig)
inv_sig

In [ ]:
dense_5 = inv_sig.eval(session=tf.compat.v1.Session())
print(dense_5)
dense_5 -= model.layers[2].get_weights()[1]
weights5 = model.layers[2].get_weights()[0]
weights5 = weights5.transpose()
print(weights5)
activation_4 = np.linalg.lstsq(weights5, dense_5, rcond=None)[0]
#activation_4 = nnls(weights5, dense_5)[0]
#skipping RELU Activation
dense_3 = activation_4
dense_3 -= model.layers[0].get_weights()[1]
weights3 = model.layers[0].get_weights()[0]
weights3 = weights3.transpose()
#gen_input = np.linalg.lstsq(weights3, dense_3, rcond=None)[0]
gen_input = nnls(weights3, dense_3)[0]

with np.printoptions(threshold = np.inf):
    print(gen_input)

In [ ]:
check = np.rint(model(gen_input.reshape(-1, MAX_FILE_SIZE)))
print(check)
print(np.rint(sample_map))

In [ ]:
# Checking result
layer_output = model.layers[1].output
inter_model = tf.keras.models.Model(inputs=model.input, outputs=layer_output)
inter_pred = inter_model(X_train[0].reshape(-1, MAX_FILE_SIZE))
inter_pred
#X_train.shape
#MAX_BITMAP_SIZE

In [ ]:
#Computing the Gradient
analyzer = innvestigate.create_analyzer("integrated_gradients", model, neuron_selection_mode="index")
a = analyzer.analyze(X_train)

#with np.printoptions(threshold = np.inf):
    #print(a[0])
    #print(X_train[0])


Ok for real now:
The plan is simple
Gradient guided mutation but based on paths that have appeared with the least frequency
Get gradient of input to specific output neuron that is covered little from the counter
then do mutation magic

If least frequent appears more than X times, then proceed to flip largest gradient overall
If least frequent counter doesnt change in y interations, flip largest gradient overall too

In [ ]:
def getSmallestPath():
    
    path = counter[-1]
    idx = label.index(int(path[0]))
    
    try:
        found = list(u_idx).index(idx)
    except ValueError:
        found = -1
        
    i = -2
    while(found == -1):
        path = counter[i]
        idx = label.index(int(path[0]))

        try:
            found = list(u_idx).index(idx)
        except ValueError:
            found = -1

        #Figure out an alternative method for here because .index crashes the program if not found
        i -= 1
        
    return found


def getLargestPath(): #Idea: Instead of trying to turn on the least visited path, avoid the most visited paths
    
    path = counter[0]
    idx = label.index(int(path[0]))
    try:
        found = list(u_idx).index(idx)
    except ValueError:
        found = -1
    
    i = 1
    while(found == -1):
        path = counter[i]
        idx = label.index(int(path[0]))
        try:
            found = list(u_idx).index(idx)
        except ValueError:
            found = -1
        
        #Figure out an alternative method for here because .index crashes the program if not found
        i += 1
        
    return found

In [ ]:
def getSmallestGradient(curr_seed):
    analyzer = innvestigate.create_analyzer("integrated_gradients", model, neuron_selection_mode="index")
    
    target_index = getSmallestPath()
    
    a = analyzer.analyze(curr_seed, target_index)

In [ ]:
#Computing the Gradient
analyzer = innvestigate.create_analyzer("integrated_gradients", model, neuron_selection_mode="index")

#Determining the index of the least frequented bit

#If an index cant be found (because it was removed by np.unique) just look at the next smallest item in the counter,
#this always works because if the index is dependant on another one, we will always stumble into the right because its count
#is the same. Otherwise if a new path is found then the bitmap gets updated and the previously missing index will appear

#target_index = getSmallestPath()

path = counter[18]
idx = label.index(int(path[0]))
target_index = list(u_idx).index(idx) #Figure out an alternative method for here because .index crashes the program if not found
#print(idx)
#print(u_idx) #Technically u_idx should be sorted in reverse order already anyways
    

#print(X_val[0]) #Replace with current fuzzing input
curr_in = np.expand_dims(X_val[0], axis=0)

a = analyzer.analyze(curr_in, target_index) #If weights are too small either mutate with overall gradient or add new inputs

with np.printoptions(threshold = np.inf):
    print(a)
    print(target_index)

In [ ]:
#Computing the Gradient
analyzer = innvestigate.create_analyzer("integrated_gradients", model)

#Determining the index of the least frequented bit

#If an index cant be found (because it was removed by np.unique) just look at the next smallest item in the counter,
#this always works because if the index is dependant on another one, we will always stumble into the right because its count
#is the same. Otherwise if a new path is found then the bitmap gets updated and the previously missing index will appear

target_index = getSmallestPath()

#print(X_val[0]) #Replace with current fuzzing input
curr_in = np.expand_dims(X_val[2], axis=0)
print(curr_in.shape)
print(MAX_FILE_SIZE)


a = analyzer.analyze(curr_in)

with np.printoptions(threshold = np.inf):
    print(a)
    print(target_index)

In [ ]:
def mut_one():
    '''
    Takes a series of inputs that were used to be fuzzed, calculates the overall gradient of each set of inputs and generates
    a new set of inputs for each that essentially flips inputs with the largest gradient
    '''

In [ ]:
def mut_assist():
    '''
    Idea for this mutator is to just send gradient information over to AFL and instead adjust the various fuzzing strategies
    within afl as a way to fuzz.
    
    i.e. chanings the perf_score in calculate score or changing bit_flip priorities
    '''

In [ ]:
def getSmallestPaths(fn, start, num_paths): #Changed input_vec to fn so that we can get the bitmap
    cnt = 0
    num_found = 0
    idxs = []
    path = counter[-1]
    #print(path)
    #print(u_idx)
    idx = label.index(int(path[0]))
    #print(idx)
    
    file_name = "./targets/" + target_dir + "/bitmaps/" + fn.split('/')[-1] + ".npy"
    bitmap = np.load(file_name)
    
    try:
        found = list(u_idx).index(idx)
        if(bitmap[found] == 0 and start < cnt):
            idxs.append(found)
            cnt += 1

    except ValueError:
        pass
        
    i = -2
    while(cnt < num_paths):
        #print(i)
        path = counter[i]
        idx = label.index(int(path[0]))
        try:
            found = list(u_idx).index(idx)
            if found != -1 and bitmap[found] == 0 and start <= num_found:
                idxs.append(found)
                cnt += 1
            if found != -1:
                num_found += 1

        except ValueError:
            pass

        #Figure out an alternative method for here because .index crashes the program if not found
        i -= 1
        
        if i <= -len(counter) or cnt >= num_paths:
            break
        #Add condition to break if i becomes greater than the size of the counter before num_paths is fufilled        
    return idxs


In [ ]:
idx_on = getSmallestPaths(seed_list[0], 50, 55)
#idx_on = tf.convert_to_tensor(idx_on)
idx_on

In [ ]:
print(X_train[0])

In [ ]:
def adv_categorical_accuracy(y_true, y_pred):
    #y_pred = tf.gather(y_pred, indices=idx_on)
    #y_true = tf.gather(y_true, indices=idx_on)
    
    return K.cast(K.equal(K.argmax(y_true, axis=-1),
                          K.argmax(y_pred, axis=-1)),
                  K.floatx())

# compute jaccard accuracy for multiple label
def adv_jaccard(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)
    y_true = tf.reshape(tf.round(y_true), [-1])
    y_pred = tf.reshape(tf.round(y_pred), [-1])
    
    y_pred = tf.gather(y_pred, indices=idx_on)
    y_true = tf.gather(y_true, indices=idx_on)
    
    matching = tf.cast(tf.math.count_nonzero(tf.equal(y_true,y_pred)), tf.float64)
    #print(matching)
    denom = tf.cast(tf.math.add(tf.size(y_true), tf.size(y_pred)), tf.float64)
    #print("mean")
    denom2 = tf.cast(tf.math.subtract(denom, matching), tf.float64)
    #print(K.mean(tf.cast(tf.divide(matching, denom2), tf.float64)))
    return tf.cast(tf.divide(matching, denom2), tf.float64)  

def hamming_loss(y_true, y_pred):
    """
    XOR TT for reference -
     
    A  B   Output
     
    0  0    0
    0  1    1
    1  0    1
    1  1    0
    """
    hl_num = np.sum(np.logical_xor(y_true, y_pred))
    hl_den = np.prod(y_true.shape)
     
    return hl_num/hl_den

def alpha_evaluation_score(y_true, y_pred):
    alpha = 1
    beta = 0.25
    gamma = 1
     
    # compute true positives across training examples and labels
    tp = tf.add(tf.logical_and(y_true, y_pred))
     
    # compute false negatives (Missed Labels) across training examples and labels
    fn = tf.add(tf.logical_and(y_true, tf.logical_not(y_pred)))
     
    # compute False Positive across training examples and labels.
    fp = tf.add(tf.logical_and(tf.logical_not(y_true), y_pred))
         
    # Compute alpha evaluation score
    alpha_score = (1 - ((beta * fn + gamma * fp ) / (tp +fn + fp + 0.00001)))**alpha
    
    return alpha_score
     
    
def adv_BinaryCrossEntropy(y_true, y_pred): 
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    y_pred = tf.gather(y_pred, indices=idx_on)
    y_true = tf.gather(y_true, indices=idx_on)
    
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    term_0 = (1 - y_true) * K.log(1 - y_pred + K.epsilon())  
    term_1 = y_true * K.log(y_pred + K.epsilon())
    return -K.mean(term_0 + term_1, axis=0)

In [ ]:


#function for generating an adversarial example given a base p_input, adversarial class target, classifier, and regularization type
def generate_adversary(p_in,target_vector,model,regularization,loss_function):
    
    #input for base p_input
    p_input = Input(shape=(1, MAX_FILE_SIZE),name='p_input')#Consider trying to reshape each of these to its transpose (MAX_FILE_SIZE, 1)

    #unit input for adversarial noise
    one = Input(shape=(1,), name='unity')
    print("ran")
    #layer for learning adversarial noise to apply to p_input
    noise = Dense(MAX_FILE_SIZE,activation = None,use_bias=False,kernel_initializer='random_normal',
                  kernel_regularizer=regularization, name='adversarial_noise')(one)
    
    #reshape noise in shape of p_input
    noise = Reshape((1, MAX_FILE_SIZE),name='reshape')(noise)
    
    #add noise to p_input
    net = Add(name='add')([noise,p_input])
    #clip values to be within 0.0 and 1.0
    net = Activation('clip', name='clip_values')(net)
    
    #feed adversarial p_input to trained model
    outputs = model(net)

    adversarial_model = Model(inputs=[p_input,one], outputs=outputs)
    #freeze trained MNIST classifier layers
    adversarial_model.layers[-1].trainable = False
    print("ran2")
    adversarial_model.compile(optimizer='adam', loss=loss_function, metrics=[adv_jaccard]) #Categorical is most likely not great for this purpose

    #callback for saving weights with smallest loss
    checkpoint = ModelCheckpoint('./targets/' + target_dir + '/checkpoint/adversarial_weights.h5', monitor='loss', verbose=0, save_best_only=True, save_weights_only=True,
                                 mode='auto', period=1)
    print("ran3")
    print(adversarial_model.summary())
    #train adversarial p_input
    adversarial_model.fit(x=[p_in.reshape(-1, 1, MAX_FILE_SIZE),np.ones(shape=(1,1))],y=target_vector.reshape(-1, 1, MAX_BITMAP_SIZE),epochs=5000,verbose=1, callbacks=[checkpoint])
    print("ran4")
    #restore best weights
    adversarial_model.load_weights('./targets/' + target_dir + '/checkpoint/adversarial_weights.h5')
    
    #quantize adversarial noise
    quantized_weights = np.round(adversarial_model.get_weights()[0].reshape((1, MAX_FILE_SIZE)) * 255.) / 255.
    
    #add trained weights to original p_input and clip values to produce adversarial p_input
    adversarial_p_in = np.clip(p_in.reshape((1,MAX_FILE_SIZE)) + quantized_weights, 0., 1.)
    
    #display adversarial p_input
    #print(p_in)
    #print(adversarial_p_in) #check what kind of p_input is fed into the network
    #plt.imshow(adversarial_p_in,vmin=0., vmax=1.)
    #plt.show()
    #classify adversarial p_input
    #adversarial_prediction = fmodel.predict(adversarial_p_in.reshape((1,28,28,1)))
    #print(adversarial_prediction)
    
    return adversarial_p_in

In [ ]:
#custom activation function for keeping adversarial values between 0.0 and 1.0
def clip(x):
    return K.clip(x, 0.0, 1.0)


def mut_adv(fn):
    '''
    Generates an aversarial example that targets less visited nodes and feeds it as input for the fuzzer to use as a seed
    '''
    #First Create a Target Vector of Values
    path = counter[0]
    idx = label.index(int(path[0]))
    zeros = []
    ones = []
    try:
        zeros.append(list(u_idx).index(idx))
    except ValueError:
        pass
    
    i = 1
    #First half of paths that are most frequented are set to 0 and second half are set to 1
    while(i <= len(counter)/2): #Thought to perhaps disclude nodes from items if execution path appears in every single generated case
        path = counter[i]
        idx = label.index(int(path[0]))
        try:
            zeros.append(list(u_idx).index(idx))
        except ValueError:
            pass
            
        #Figure out an alternative method for here because .index crashes the program if not found
        i += 1

    while(i < len(counter)):
        path = counter[i]
        idx = label.index(int(path[0]))
        try:
            ones.append(list(u_idx).index(idx))
        except ValueError:
            pass

        i += 1
        
    
    target_vector = np.zeros(MAX_BITMAP_SIZE)
    target_idx = getSmallestPaths(fn, 0, 10)
    
    for i in target_idx:
        target_vector[i] = 1

    #add custom objects to dictionary
    get_custom_objects().update({'clip': clip})
    #print(target_vector)
    #print(target_vector.reshape(1, -1))


    p_in = np.random.normal(.5, .3, (1, MAX_FILE_SIZE))
    p_in,bit = vectorize(fn)

    
    adv_jaccard(model.predict(p_in).flatten(), target_vector)
    
    #Instead of using a random vector pick one from a previous set that maximizes the gradient?
    
    print(p_in)
    #generate_adversary(img,9,model,l1(0.01),'categorical_crossentropy')
    
   # with CustomObjectScope({'clip': Activation(clip)}):
    adv = generate_adversary(p_in,target_vector,model,l2(0.001),adv_BinaryCrossEntropy) #greater the value the more subtle the noise
    #pred(generate_adversary(img,9,model,l1_l2(l1=0.01,l2=0.01),'categorical_crossentropy')
    
    return adv, target_vector
    

In [ ]:
    
#loss_func = tf.keras.losses.CosineSimilarity

adv_in, target_vector = mut_adv(seed_list[0])

In [ ]:
print(adv_in)
pred = model.predict(adv_in)
with np.printoptions(threshold = np.inf):
    r_pred = np.rint(pred).flatten()
    for i in idx_on:
        print(r_pred[i])
        
    print(r_pred)
    print(target_vector)

In [ ]:
    
#loss_func = tf.keras.losses.CosineSimilarity

adv_in, target_vector = mut_adv(seed_list[0])

In [ ]:
model.trainable = False

    
target_vector = np.zeros(MAX_BITMAP_SIZE)
target_idx = getSmallestPaths(fn, 0, 10)

for i in target_idx:
    target_vector[i] = 1


loss_object = adv_BinaryCrossEntropy#tf.keras.losses.Categorical_Crossentropy()
def create_adversarial_pattern(input_image, input_label):
  with tf.GradientTape() as tape:
    tape.watch(input_image)

    #forward propagate image and retrieve the loss
    prediction = pretrained_model(input_image)
    loss = loss_object(input_label, prediction)

   # obtain the gradient of the loss with respect to the input image
   gradient = tape.gradient(loss, input_image)

   # Get the sign of the gradients (directions)
   signed_grad = tf.sign(gradient)
   return signed_grad

epsilons = [0, 0.01, 0.1, 0.15]
descriptions = [('Epsilon = {:0.3f}'.format(eps) if eps else 'Input')
                for eps in epsilons]
for i, eps in enumerate(epsilons):
  adv_x = image - eps*perturbations #input_image - epsilon * gradients
  adv_x = tf.clip_by_value(adv_x, -1, 1)
  adv_jaccard(model.predict(adv_x), )

In [ ]:
print(idx_on)
def adv_jaccard(y_pred, y_true):
    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)
    y_true = tf.reshape(tf.round(y_true), [-1])
    y_pred = tf.reshape(tf.round(y_pred), [-1])
    
    print(tf.keras.backend.get_value(y_pred))
    print(tf.keras.backend.get_value(y_true))
    
    y_pred = tf.gather(y_pred, indices=idx_on)
    y_true = tf.gather(y_true, indices=idx_on)
    
    print(tf.keras.backend.get_value(y_pred))
    print(tf.keras.backend.get_value(y_true))
    
    matching = tf.cast(tf.math.count_nonzero(tf.equal(y_true,y_pred)), tf.float64)
    #print(matching)
    denom = tf.cast(tf.math.add(tf.size(y_true), tf.size(y_pred)), tf.float64)
    #print("mean")
    denom2 = tf.cast(tf.math.subtract(denom, matching), tf.float64)
    #print(K.mean(tf.cast(tf.divide(matching, denom2), tf.float64)))
    return tf.cast(tf.divide(matching, denom2), tf.float64)  




In [ ]:
target_vector = np.zeros(MAX_BITMAP_SIZE)
target_idx = getSmallestPaths(seed_list[0], 0, 5)
print(target_idx)

for i in target_idx:
    target_vector[i] = 1

#add custom objects to dictionary
get_custom_objects().update({'clip': clip})
#print(target_vector)
#print(target_vector.reshape(1, -1))


p_in = np.random.normal(.5, .3, (1, MAX_FILE_SIZE))
p_in,bit = vectorize(seed_list[0])


adv_jaccard(model.predict(p_in).flatten(), target_vector)

In [ ]:
#Gradients of no input are removed from consideration because they are a) small and b) mean nothing to the fuzzer for mutating inputs
def compAbsIndexGradientMean(file_name, tar_index):
    
    curr_in, res = vectorize(file_name)

    print(curr_in.shape)
    #Computing the Gradient
    analyzer = innvestigate.create_analyzer("integrated_gradients", model, neuron_selection_mode="index")

    a = analyzer.analyze(curr_in, tar_index)
    a = a[a != 0]
    
    mean = np.mean(np.absolute(a))
    fname = file_name.split('/')[5]
    fname = "./targets/"  + target_dir + "/out/queue_grads/" + fname + "_" + str(tar_index) + ".txt"
    with open(fname, "w") as file:
       file.write(str(f"{mean:8f}"))
    
    return mean

In [ ]:
compAbsIndexGradientMean(seed_list[2], 40)

In [ ]:
#Computes average gradient of input
def compAbsGradientMean(file_name):
    
    curr_in, res = vectorize(file_name)

    print(curr_in.shape)
    #Computing the Gradient
    analyzer = innvestigate.create_analyzer("integrated_gradients", model)

    a = analyzer.analyze(curr_in)
    a = a[a != 0]
    
    mean = np.mean(np.absolute(a))
    fname = file_name.split('/')[5]
    fname = "./targets/"  + target_dir + "/out/queue_grads/" + fname + ".txt"
    with open(fname, "w") as file:
       file.write(str(f"{mean:8f}"))
    
    return mean

In [ ]:
compAbsGradientMean(seed_list[0])

In [ ]:
print(X_train[0].shape)

In [ ]:
def connectToAfl():
        """ TCP Socket """
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect(ADDR)
    print("Connection Success")

    """ Recv data """
    data = client.recv(SIZE).decode(FORMAT)
    print(f"[SERVER] {data}")

    """ Send data """
    data += " From CLIENT"
    client.send(data.encode(FORMAT))

    """ Close connection """
    client.close()